# Model 


# import data

In [213]:
import os
import sys
from pyprojroot import here

import pandas as pd

In [214]:
# Read in the data in the previous step
df = pd.read_pickle('..\data\processed\data_clean.pkl')
df.T

,0,1,2,3,4,5,6,7,8,9,...,3124,3125,3126,3127,3128,3129,3130,3131,3132,3133
project_title,density matrix renormalization group simulatio...,adolescent depressive vulnerability through th...,autonomic computing laboratory,laboratory for forest genomics,stoe ipds diffractometer stateoftheart image ...,infrastructure to investigate the role of t ly...,malleable costeffective highperformance comput...,cold regions biogeochemical research,sexual health research facility,role of vagal afferents in nutrient signalling...,...,gene therapy development for musculoskeletal d...,surviving memory in postwar el salvador a coll...,mapping stories global research lab,speciation analyses for elucidating molecular ...,the climatedatadriven design cd3 facility for ...,establishment of the intelligent visualization...,infrastructure for the catalytic synthesis of ...,niagara college nextgen greenhouse excellence,lambton water centre,cambrian college
project_description,proposed infrastructure to consist of a comput...,the proposed infrastructure consisting of onsi...,autonomic computing laboratory will enable res...,the proposed facility will enable the examina...,researchers at mcmaster university intend to u...,allergic and autoimmune diseases such as asthm...,a malleable costeffective highperformance comp...,the proposed research will investigate the bio...,the research infrastructure is required for th...,the objective of the research is to understand...,...,this proposal will identify diseasemodifying g...,surviving memory in postwar el salvador is an ...,his application requests infrastructure to est...,selenium se is an important contaminant releas...,this proposal outlines the equipment necessary...,this application is for funds to develop the i...,the proposed project will develop efficient co...,niagara college looks to expand upon strategic...,through this orf request lambton college will ...,through the battery electric vehicle lab bevl ...
area_primary,763,4,25,63,76,41,76,310,4,49,...,41,87,87,34,23,10,74,NaN,NaN,NaN
discipline_primary,12300.0,15100.0,11800.0,14400.0,13000.0,15200.0,11600.0,13500.0,15100.0,15200.0,...,41800.0,20001.0,22700.0,12900.0,10100.0,11800.0,13100.0,NaN,NaN,NaN
approval_date,2005-10-24 00:00:00,2005-07-22 00:00:00,2005-10-24 00:00:00,2005-07-22 00:00:00,2006-12-21 00:00:00,2006-08-31 00:00:00,2005-07-22 00:00:00,2005-10-24 00:00:00,2005-10-24 00:00:00,2005-10-24 00:00:00,...,2021-12-17 00:00:00,2021-12-17 00:00:00,2021-12-17 00:00:00,2021-12-17 00:00:00,2021-12-17 00:00:00,2021-12-17 00:00:00,2021-12-17 00:00:00,2022-03-01 00:00:00,2022-03-01 00:00:00,2022-03-01 00:00:00
lead_research_institution,university of windsor,university of toronto,university of toronto,university of toronto,mcmaster university,mcmaster university,queens university,queens university,queens university,queens university,...,western university,western university,wilfrid laurier university,wilfrid laurier university,york university,york university,york university,niagara college,lambton college,cambrian college
city,windsor,toronto,toronto,toronto,hamilton,hamilton,kingston,kingston,kingston,kingston,...,london,london,waterloo,waterloo,toronto,toronto,toronto,niaraga on the lake,sarnia,sudbury
ontario_commitment,50000,59422,292551,200000,100000,197078,100000,128000,64000,125000,...,387254,52026,57077,200000,140000,114726,160000,892391,597715,1000000
total_project_costs,175708,148557,731377,500000,261369,492695,441005,320000,160000,312500,...,968135,130067,150858,500000,365313,329452,447211,7422302,1494288,2765734
keyword,density matrix renormalization group numerical...,depression adolescence cognitive vulnerability...,scaling availability adaptive selfmanaging sel...,comparative functional genomics molecular biol...,magnetocaloric materials for magnetic refriger...,immunology immunomodulation asthma allergy aut...,reconfigurable architectures selfadaptive comp...,biogeochemistry arctic and alpine environments...,sexual health laboratory p

# Add a prediction column y

In [215]:
df.total_project_costs.min(),df.total_project_costs.max(),df.total_project_costs.mean()

(11852, 58435136, 1473472.2000638163)

In [216]:
df['total_project_costs'] > df.total_project_costs.mean()

0       False
1       False
2       False
3       False
4       False
        ...  
3129    False
3130    False
3131     True
3132     True
3133     True
Name: total_project_costs, Length: 3134, dtype: bool

In [217]:
df["y"] = (df['total_project_costs'] > df.total_project_costs.mean()).astype(int)

# Split the data to train-test

In [218]:
from sklearn.model_selection import train_test_split
import pandas as pd

In [219]:
x=df.drop(['y'],axis=1)
y=df['y']

In [220]:
x_train,x_test,y_train,y_test =train_test_split(x,y)

In [221]:
len(x_train),len(x_test)

(2350, 784)

In [222]:
x_train.shape,y_train.shape

((2350, 12), (2350,))

In [223]:
x_train['project_title']

704          broadband communications networks laboratory
2443    printing adaptive aerodynamic and aeroelastic ...
2846    high resolution nmr facility for structure elu...
1365    socioeconomic status social networks and cardi...
1021    laboratory for the study of polarity proteins ...
                              ...                        
2890    neural substrates of cognitive influence over ...
1263        laboratory for multifunctional nanocomposites
28                          psychological trait evolution
2472    realtime imaging platforms for monitoring canc...
628     integrative computational biology towards inte...
Name: project_title, Length: 2350, dtype: object

# Tekens

In [86]:
#tokens=[]
#for text in x_train['project_title']:
#    tokens.append(list(set(text.split())))

In [225]:
#tokens[2345:2346]

In [224]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Atieh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [272]:
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from nltk.corpus import stopwords

In [267]:
tfidfvectorizer = TfidfVectorizer(analyzer='word',
                             lowercase=True,
                             max_df=0.9,
                             min_df=2,
                             ngram_range=(1,3),
                             stop_words='english')

In [268]:
tfidfvectorizer.fit(x_train['project_title'])
tfidf_train = tfidfvectorizer.transform(x_train['project_title'])

In [269]:
tfidf_test  = tfidfvectorizer.transform(x_test['project_title'])

In [274]:
countvectorizer = CountVectorizer(analyzer= 'word', stop_words='english')
count_wm=countvectorizer.fit_transform(x_train['project_title'])

In [278]:
count_tokens = countvectorizer.get_feature_names()
tfidf_tokens = tfidfvectorizer.get_feature_names()

C:\Users\Atieh\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [279]:
df_countvect = pd.DataFrame(data = count_wm.toarray(),columns = count_tokens)
df_tfidfvect = pd.DataFrame(data = tfidf_train.toarray(),columns = tfidf_tokens)
print("Count Vectorizer\n")
print(df_countvect)
print("\nTD-IDF Vectorizer\n")
print(df_tfidfvect)

Count Vectorizer

      1000mhz  1861  1871  18752010  1d  20  2020  21st  2d  2d3d  ...  youth  \
0           0     0     0         0   0   0     0     0   0     0  ...      0   
1           0     0     0         0   0   0     0     0   0     0  ...      0   
2           0     0     0         0   0   0     0     0   0     0  ...      0   
3           0     0     0         0   0   0     0     0   0     0  ...      0   
4           0     0     0         0   0   0     0     0   0     0  ...      0   
...       ...   ...   ...       ...  ..  ..   ...   ...  ..   ...  ...    ...   
2345        0     0     0         0   0   0     0     0   0     0  ...      0   
2346        0     0     0         0   0   0     0     0   0     0  ...      0   
2347        0     0     0         0   0   0     0     0   0     0  ...      0   
2348        0     0     0         0   0   0     0     0   0     0  ...      0   
2349        0     0     0         0   0   0     0     0   0     0  ...      0   

      yuk

In [280]:
tfidf_train.shape, tfidf_test.shape

((2350, 2767), (784, 2767))

In [242]:
#tfidf.nonzero()[1]

In [245]:
#fit change and fit the model
#transform just return the value of known model
type(tfidf_train)

scipy.sparse.csr.csr_matrix

In [281]:
words=tfidfvectorizer.get_feature_names()

C:\Users\Atieh\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [282]:
len(words)

2767

In [283]:
words

['20',
 '2d',
 '3d',
 '3d lab',
 '3dimensional',
 '5g',
 '600',
 '600 mhz',
 'aboriginal',
 'acceleration',
 'accelerator',
 'access',
 'accessories',
 'acdc',
 'achieving',
 'acid',
 'acoustic',
 'acquired',
 'acquired brain',
 'acquisition',
 'action',
 'activation',
 'active',
 'activities',
 'activity',
 'activitybased',
 'acute',
 'adaptation',
 'adaptive',
 'addiction',
 'addictions',
 'additive',
 'additive manufacturing',
 'adult',
 'adults',
 'advance',
 'advanced',
 'advanced biomedical',
 'advanced building',
 'advanced characterization',
 'advanced design',
 'advanced digital',
 'advanced energy',
 'advanced imaging',
 'advanced infrastructure',
 'advanced manufacturing',
 'advanced manufacturing robotic',
 'advanced materials',
 'advanced research',
 'advanced scanning',
 'advanced sensor',
 'advances',
 'advancing',
 'aerial',
 'aerosol',
 'aerospace',
 'affective',
 'age',
 'agents',
 'aging',
 'agricultural',
 'agriculture',
 'agrifood',
 'ai',
 'aided',
 'air',
 'airbo

In [289]:
tfidf_train.nonzero()[1] #Return the indices of the elements that are non-zero.

array([1797, 1794, 1404, ...,  501,  500,  228], dtype=int32)

In [290]:
tfidf_train.nonzero()

(array([   0,    0,    0, ..., 2349, 2349, 2349], dtype=int32),
 array([1797, 1794, 1404, ...,  501,  500,  228], dtype=int32))

In [299]:
a=np.array([[3,0,4],[4,7,0]])
a, a.nonzero()

(array([[3, 0, 4],
        [4, 7, 0]]),
 (array([0, 0, 1, 1], dtype=int64), array([0, 2, 0, 1], dtype=int64)))

In [300]:
a.nonzero()[1]

array([0, 2, 0, 1], dtype=int64)

In [287]:
tfidf_train.shape

(2350, 2767)

In [304]:
m=0
for col in tfidf_train.nonzero()[1]:
    if tfidf_train[tfidf_train.nonzero()[0][m],col] > 0.4:
        print (words[col], '-', tfidf_train[tfidf_train.nonzero()[0][m],col])
        m=m+1
    else:
        m=m+1

networks laboratory - 0.5592018220044753
communications - 0.47135406463472995
broadband - 0.5219294805582185
test - 0.5803463654747112
models - 0.5253952054020035
adaptive - 0.6222201975353094
structure elucidation - 0.44226136717179315
elucidation - 0.44226136717179315
social networks - 0.5349531048401983
cardiovascular disease - 0.466490322212188
polarity - 0.47963263214517143
breast cancer - 0.41824971028015223
breast - 0.41087853241966266
photons - 0.42595777950214353
electrons - 0.42595777950214353
laboratory complex - 0.49737589405638594
obesity research - 0.44540245915350674
childhood obesity - 0.44540245915350674
trait - 0.4928465659646724
scales - 0.4928465659646724
phenomics - 0.4928465659646724
york - 0.4616601810509427
vehicles - 0.41692491212727045
unmanned - 0.47606172094981436
autonomous - 0.41692491212727045
research facility - 0.5217741398668219
nanophotonics - 0.7314879621980384
strategies - 0.48350983413660326
stem cells - 0.4483845346631784
regenerative - 0.48350983

affective - 0.4471816303086799
new - 0.5229559395256839
higherorder - 0.8523597159150643
genome sequencing - 0.41920429148120586
disease risk - 0.41920429148120586
materials development - 0.402316626647223
deposition - 0.402316626647223
cell - 0.44655223054115767
new - 0.466067853674558
metal - 0.5608429045538449
imaging laboratory - 0.6068852994313342
synthesis - 0.48093993981605776
structural - 0.5097121321984925
dna - 0.5140613134615796
neural circuits - 0.5286624254115767
neural - 0.40402716025423246
circuits - 0.45804424691961987
source - 0.5376578663795161
nonlinear - 0.585793470819223
laser - 0.5075437865343428
surfaces - 0.5472443405713041
smart - 0.47221466456705363
nanoscale - 0.5289724234001039
investigation - 0.4446629257794323
space - 0.43115196500109354
instrument - 0.4549570489249553
developments - 0.5115071290968065
astronomy - 0.4923067451211901
microfluidics - 0.7410678757254018
crc - 0.6714301181566407
longterm - 0.5438196013786698
environmental change - 0.5438196013

interactive - 0.41703844845009386
studies - 0.44454151247327167
preclinical - 0.610262269533973
aging - 0.6024812148589509
peoples - 0.8338339205108597
canadian - 0.5520153920005214
manipulation - 0.44458577454667286
imaging facility - 0.5427630995367704
facility study - 0.4507479176499932
workstation - 0.6252296867461767
advanced imaging - 0.6252296867461767
sound - 0.7085596596679227
patterns - 0.6613322070185017
temperature - 0.4792907376744104
pressure - 0.5135181356123896
planetary - 0.4399066364346912
extreme - 0.4328470526381996
world - 0.4017866796227041
food security - 0.4417384559982314
pathology - 0.5499977767084774
cardiovascular disease - 0.5392239241139453
cardiovascular - 0.4325793193806291
stress health - 0.5452252599317279
stress - 0.4177572807509731
intervention - 0.435636099231969
institute - 0.4595731492596788
international - 0.4558301591919732
health communication - 0.4883821762129962
archive - 0.4883821762129962
strategic - 0.4536003622091647
individual - 0.439878

primate - 0.5718246007315849
endocrinology - 0.5503600867958034
behavioral - 0.5201075535416863
synthesis - 0.41098179991985145
organic - 0.45166501910725937
design synthesis - 0.5837965845454092
facility integrative - 0.4016264868966338
movement - 0.4052868082131772
scattering - 0.4490496209888303
neutron scattering - 0.4665629401390038
neutron - 0.424365984007078
future - 0.4149817007598711
modeling - 0.5483022865089137
measurement - 0.5621049542302716
3d - 0.5309175370129348
sustainable energy - 0.42685226010304583
management systems - 0.4573348899952128
facility research - 0.406773865087739
stars - 0.45545101011837646
new physics - 0.45545101011837646
neutrinos - 0.45545101011837646
proton - 0.42705249206095625
measurements proton - 0.42705249206095625
development novel - 0.4110222720329527
flexible - 0.43927750896564055
sequences systems - 0.4153244282251025
sequences - 0.4153244282251025
regulatory sequences systems - 0.4153244282251025
regulatory sequences - 0.4153244282251025
p

history - 0.4329044294616646
wine - 0.4358808883299359
fermentation - 0.45288061564731674
expanded - 0.45288061564731674
consumer - 0.4226948754072524
vascular - 0.5757044250136413
transcriptional - 0.6168169750812856
regulation - 0.4430161706750819
techniques - 0.4475926377095508
motion - 0.43264798685371925
joint - 0.43264798685371925
hybrid - 0.41507659701779864
treeline - 0.4506345433019745
infrastructure analysis - 0.4506345433019745
hydrology - 0.4098781856504676
changes - 0.4098781856504676
arctic - 0.4008142805874947
ultrasound imaging - 0.4226259486262803
imaging spectroscopy - 0.4226259486262803
advanced biomedical - 0.4226259486262803
materials laboratory - 0.7089625765680402
materials - 0.40182236102525143
biological - 0.524651327033992
cryoelectron - 0.5178121606269119
complexes - 0.47098011472964946
organisms - 0.4240692612870247
ecological evolutionary - 0.44500135863338125
welding - 0.7775687976530377
laser - 0.6287978728625038
research laboratory - 0.7744243873220559
r

...

brain circuits - 0.5287866802689453
technology laboratory - 0.5540986333168436
infant - 0.5171664334505057
hearing - 0.5171664334505057
health aging - 0.41610871121442544
research biomatrix - 0.4236760366423901
expanded - 0.4236760366423901
cancer research biomatrix - 0.4236760366423901
biomatrix - 0.4077725582311212
industry - 0.4419647271781348
enhancing - 0.41883945096420894
collaboration - 0.43330712341272226
analytics - 0.40182887540352025
testing facility - 0.47520002117352655
microfluidic - 0.42266390238574225
based - 0.4143843805873344
performance laboratory - 0.580731483595028
performance - 0.40996328202307875
distributed - 0.5261604522606154
digital - 0.41701838959178994
study plant - 0.47078279455758815
carleton - 0.4531110752199117
pathways - 0.7058902023101484
disease - 0.4633284706174086
brain - 0.5357665075365114
microvascular - 0.5607778986307225
gastrointestinal - 0.5100599389461745
dysfunction - 0.46570864947832147
protein dynamics - 0.5273629482822441
membrane protei

assessment intervention - 0.6269162800762812
assessment - 0.44420773441804523
source - 0.5812491134265777
photon - 0.6785120667246022
canadian - 0.4491890954263322
management - 0.4252193664305886
grid - 0.5775911935569437
edge - 0.5775911935569437
states - 0.6253817233931085
regulation - 0.46668554733538853
elegans - 0.6253817233931085
interactive - 0.8758960487474658
facility - 0.48249985677570617
nanofabrication - 0.7071067811865476
metamaterials - 0.7071067811865476
realtime - 0.4598821431815689
observation - 0.4852734991353679
ecosystem - 0.4598821431815689
aquatic - 0.43101746103392563
insects - 0.4830734596442836
changing climate - 0.45087528320246545
changing - 0.41382612617870557
tools - 0.4189361523799425
multiscale - 0.5282005907786674
modelling - 0.5050558469828378
design - 0.41023009826130663
verification - 0.47410637796444927
iot - 0.4925969320598746
adaptation - 0.4090864717153112
making - 0.5712839504916722
information - 0.4792128799474868
digital - 0.4439123694241451
cr

metabolic - 0.44672602449899657
characterization laboratory - 0.6068044827056178
chemical biology - 0.5127841999826279
biology research - 0.532783222205799
loss - 0.408174813720277
global environmental - 0.4240939804395462
network - 0.40484365573666353
manufacturing - 0.42488507402619435
additive manufacturing - 0.5090622994034003
additive - 0.5090622994034003
target - 0.4965456860461077
leukemia - 0.5159113923420354
acute - 0.481524477252134
structurefunction - 0.4328878269165043
stateoftheart facility - 0.41663856659858206
relationships - 0.404034661104339
society - 0.4859068761007265
research chair - 0.4175375099713352
contemporary - 0.5141700955589751
loads - 0.5536384254693641
cyclic - 0.5536384254693641
behavior - 0.4827843135210924
pollutants - 0.41393208536352666
development nextgeneration - 0.4300757905992753
metabolic - 0.49712222436199566
evaluation - 0.491938572991825
emerging - 0.5052004983571248
diseases - 0.4504849721793528
bacterial - 0.5475693441958265
research develop

performance - 0.4440899385532834
multidisciplinary - 0.559914838025177
measurements - 0.5353804363228389
perception - 1.0
properties - 0.49258834315288125
innovative - 0.4260141033034224
enhanced - 0.49996475961135084
antimicrobial - 0.5708799761099039
support - 0.41160090502062713
infrastructure support - 0.4583394908344049
immunology - 0.591403376375519
laboratory - 0.4061450417694116
applied - 0.9138086260514959
treatments - 0.443971569380075
phenomics - 0.5069446876979836
conditions - 0.443971569380075
synthetic - 0.4786195378731564
macromolecular - 0.5064588826144116
advanced materials - 0.44354611143446415
new - 0.5367677796317926
environment - 0.6374956966517685
advanced - 0.4616483225674027
writing - 0.5283123321093617
research collaboratory - 0.5283123321093617
collaboratory - 0.5283123321093617
selfassembly - 0.47681113511248124
highresolution - 0.4240968983333119
advanced materials - 0.40190653491212835
singlecell - 0.4226661559216064
probing - 0.40680058528621316
cellular n

cardiovascular disease - 0.465663894111213
tomography - 0.4698558162440139
ion - 0.45788000887944513
focused - 0.4845130198801412
beam - 0.5034094419078226
operation - 0.43694520297810635
involving - 0.45057575642974873
hydrogen - 0.45057575642974873
electrochemical - 0.41639205331616025
substrates - 0.411222946633666
nanocomposites - 0.7343174962183817
multifunctional - 0.6290549786549324
trait - 0.8014436977894035
evolution - 0.5980702293825094
integrative computational biology - 0.4172792792120488
integrative computational - 0.4172792792120488
computational biology - 0.4016159151449891


In [305]:
m=0
for col in tfidf_test.nonzero()[1]:
    if tfidf_test[tfidf_train.nonzero()[0][m],col] > 0.4:
        print (words[col], '-', tfidf_test[tfidf_train.nonzero()[0][m],col])
        m=m+1
    else:
        m=m+1

testing - 0.4214827579555039
materials characterization - 0.5852331492315789
developing - 0.46782220707592603
landscape - 0.7514845041925406
infrastructure analysis - 0.8223639196912973
analysis - 0.44826960930168064
receptor - 0.5844475424452459
equipment - 0.4168894493359905
building - 0.40040093579555147
spectrometry - 0.4221406276506199
xray - 0.4279181563084592
spinal - 0.47448164377901514
computational - 0.522560760100978
nmr - 0.5649313437312399
analysis - 0.432037921211762
smart - 0.48325416355121525
laboratory - 1.0
regulation - 0.45569451062052385
functional - 0.5766866696116001
laboratory - 0.6318951855793193
assessing - 0.5264619818713324
environment - 0.5320508485779029
centre - 0.4280015922996025


# Model

In [208]:
x_train=tfidf

In [209]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(x_train,y_train)

LogisticRegression()

In [210]:
x_test.shape,x_train.shape

((784, 12), (2350, 2790))

In [211]:
y_test.shape,y_train.shape

((784,), (2350,))

In [69]:
vectorizer = TfidfVectorizer(min_df=2)
tfidf_test = vectorizer.fit_transform(x_test['project_title'])

In [76]:
x_test=tfidf_test.toarray()
x_test.shape

(784, 771)

In [74]:
model.score(x_test,y_test)

ValueError: X has 771 features, but LogisticRegression is expecting 1694 features as input.